#  second step replacement

In [1]:
from datasets import load_dataset #import public datasets
from tqdm import tqdm # for jupyter view
import csv #for handle csv type

In [2]:
import flair #NLP
from flair.data import Sentence #typcial class in flair stands for sentence
from flair.models import SequenceTagger #use for NER

# load tagger
tagger = SequenceTagger.load("flair/ner-english-large") # load a pre-trian NER moduel

2024-03-10 15:15:52,568 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [3]:
"""
#找不到anoanonymizer库
import anonymizer
from anonymizer import entity
from anonymizer.core import initialize
from anonymizer.cache import NECache
import anonymizer.entity.person as person
from anonymizer.entity.org import org, org_wiki
from anonymizer.entity.gpe import gpe, gpe_wiki
from functools import partial
initialize()
"""

'\n#找不到anoanonymizer库\nimport anonymizer\nfrom anonymizer import entity\nfrom anonymizer.core import initialize\nfrom anonymizer.cache import NECache\nimport anonymizer.entity.person as person\nfrom anonymizer.entity.org import org, org_wiki\nfrom anonymizer.entity.gpe import gpe, gpe_wiki\nfrom functools import partial\ninitialize()\n'

In [4]:
''' #找不到anonymizer的库函数，所以重写一个
def replace_entities_flair_wiki(text):
    # make example sentence
    sentence = Sentence(text)
    # predict NER tags
    tagger.predict(sentence)
    # iterate over entities and print
    replacements = [] #存储待替换实体信息
    replacement_map = {}
    #实体映射
    if not sentence.get_spans('ner'):
        return text

    for entity in sentence.get_spans('ner'):
        if entity.text in replacement_map: ##已经在实体映射中有了
            replacements.append((entity.start_position, entity.end_position, replacement_map[entity.text], entity.text))
            ##存储替换信息
            continue
        if entity.get_label().value == "ORG":
            repl = org.handle(entity.text.split(" "), NECache()) ##从Wiki里面获取一个组织
            if not repl or " ".join(repl) == entity.text:##为空或者和原来文本一样
                repl = org.handle(entity.text.split(" "), NECache())##重新选取一个
            if not repl or " ".join(repl) == entity.text:
                continue  ##再次替换失败，则放弃这个
            replacements.append((entity.start_position, entity.end_position, " ".join(repl), entity.text))
            ##存储替换信息
            replacement_map[entity.text] = " ".join(repl)
            ##存储实体映射
        elif entity.get_label().value == "PER":
            repl = person.handle(entity.text.split(" "), NECache())
            if not repl or " ".join(repl) == entity.text:
                repl = person.handle(entity.text.split(" "), NECache())
            if not repl or " ".join(repl) == entity.text:
                continue
            replacements.append((entity.start_position, entity.end_position, " ".join(repl), entity.text))
            replacement_map[entity.text] = " ".join(repl)
        elif entity.get_label().value == "LOC":
            repl = gpe.handle([entity.text], {})
            if not repl or " ".join(repl) == entity.text:
                repl = gpe.handle([entity.text], {})
            if not repl or " ".join(repl) == entity.text:
                continue
            replacements.append((entity.start_position, entity.end_position, " ".join(repl), entity.text))
            replacement_map[entity.text] = " ".join(repl)

    #完成替换，将识别替换后的文本重新组合成一个完整的字符串
    if replacements:
        res = []
        i = 0
        s = text
        for (start, end, txt, orig) in replacements:
            assert orig != txt
            res.append(s[i:start] + txt)
            i = end
        res.append(s[end:])
        return ''.join(res)
    return text
'''

' #找不到anonymizer的库函数，所以重写一个\ndef replace_entities_flair_wiki(text):\n    # make example sentence\n    sentence = Sentence(text)\n    # predict NER tags\n    tagger.predict(sentence)\n    # iterate over entities and print\n    replacements = [] #存储待替换实体信息\n    replacement_map = {}\n    #实体映射\n    if not sentence.get_spans(\'ner\'):\n        return text\n\n    for entity in sentence.get_spans(\'ner\'):\n        if entity.text in replacement_map: ##已经在实体映射中有了\n            replacements.append((entity.start_position, entity.end_position, replacement_map[entity.text], entity.text))\n            ##存储替换信息\n            continue\n        if entity.get_label().value == "ORG":\n            repl = org.handle(entity.text.split(" "), NECache()) ##从Wiki里面获取一个组织\n            if not repl or " ".join(repl) == entity.text:##为空或者和原来文本一样\n                repl = org.handle(entity.text.split(" "), NECache())##重新选取一个\n            if not repl or " ".join(repl) == entity.text:\n                continue  ##再次替换失败

In [15]:
cls_data = load_dataset("imdb")
train_data = cls_data['train']
print(train_data[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [19]:
import random
from SPARQLWrapper import SPARQLWrapper, JSON

# 定义一个全局变量来存储已查询过的实体及其类别信息
cached_entities = {}

def query_wikidata(entity_text, entity_type):
    """
    通过SPARQL查询Wikidata,获取指定实体的标准标签
    如果存在多个标签,随机选择一个作为替换文本
    entity_text: 实体文本
    entity_type: 实体类型,如"ORG"、"PER"等
    """
    # 如果实体信息已经被缓存过，直接返回缓存的结果
    if entity_text in cached_entities:
        return cached_entities[entity_text]

    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")  # 初始化SPARQL查询对象
    query_template = """
SELECT ?item ?itemLabel
WHERE {{
    ?item wdt:P31 wd:%s. # 匹配实体类型
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }} # 使用英文标签
}}
LIMIT 1000
"""

    query = query_template % entity_type_to_qid(entity_type)  # 格式化查询语句
    sparql.setQuery(query)  # 设置查询语句
    sparql.setReturnFormat(JSON)  # 设置返回JSON格式
    results = sparql.query().convert() # 查询并解析结果

    if results["results"]["bindings"]:  # 如果查询结果不为空
        # 构建一个包含所有候选标签的列表
        candidate_labels = [binding["itemLabel"]["value"] for binding in results["results"]["bindings"]]
        if len(candidate_labels) > 1:  # 如果存在多个候选标签
            # 随机选择一个作为替换文本
            repl = random.choice(candidate_labels)
        else:
            repl = candidate_labels[0]  # 只有一个候选标签,直接返回

        # 将查询结果存入缓存
        cached_entities[entity_text] = repl
    else:
        repl = entity_text  # 查询失败则返回原始实体文本

    return repl

def entity_type_to_qid(type):
    """
    将实体类型映射到Wikidata的QID (Query ID)
    """
    mapping = {
        "ORG": "Q43229",  # 组织机构
        "PER": "Q5",      # 人物
        "LOC": "Q618123"  # 地点
    }
    return mapping.get(type, None)  # 返回映射的QID，如果类型不存在则返回None

In [20]:
def replace_entities_flair_wiki(text):
    sentence = Sentence(text)  # 创建一个Sentence对象
    tagger.predict(sentence)  # 使用预定义的tagger预测命名实体
    replacements = []  # 存储需要替换的实体信息
    replacement_map = {}  # 存储已替换的实体映射

    if not sentence.get_spans('ner'):  # 如果没有命名实体
        return text  # 直接返回原始文本

    for entity in sentence.get_spans('ner'):  # 遍历每个命名实体
        if entity.text in replacement_map:  # 如果实体已被替换过
            # 将替换信息添加到replacements
            replacements.append((entity.start_position, entity.end_position, replacement_map[entity.text], entity.text))
            continue  # 继续处理下一个实体

        # 根据实体类型查询Wikidata获取替换文本
        if entity.get_label().value == "ORG":
            repl = query_wikidata(entity.text, "ORG")
        elif entity.get_label().value == "PER":
            repl = query_wikidata(entity.text, "PER")
        elif entity.get_label().value == "LOC":
            repl = query_wikidata(entity.text, "LOC")
        else:
            continue  # 对其他类型实体不进行替换

        # 如果查询结果不为空且不等于原实体文本
        if repl and repl != entity.text:
            # 将替换信息添加到replacements和replacement_map
            replacements.append((entity.start_position, entity.end_position, repl, entity.text))
            replacement_map[entity.text] = repl

    if replacements:  # 如果有需要替换的实体
        res = []  # 存储替换后的文本
        i = 0  # 当前位置指针
        s = text  # 原始文本
        for start, end, txt, orig in replacements:  # 遍历每个替换信息
            assert orig != txt  # 确保替换文本与原文本不同
            res.append(s[i:start] + txt)  # 将位置指针前的原文本加上替换文本
            i = end  # 更新位置指针
        res.append(s[end:])  # 添加最后一段原文本
        return ''.join(res)  # 将替换后的文本拼接并返回

    return text  # 如果没有需要替换的实体,返回原始文本

In [23]:
text='It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is about a detective agency where the detectives and clients get romantically involved with each other. Five years later, Bogdanovich\'s ex-girlfriend, Cybil Shepherd had a hit television series called "Moonlighting" stealing the story idea from Bogdanovich. Of course, there was a great difference in that the series relied on tons of witty dialogue, while this tries to make do with slapstick and a few screwball lines.<br /><br />Bottom line: It ain\'t no "Paper Moon" and only a very pale version of "What\'s Up, Doc".'
src=replace_entities_flair_wiki(text.replace("<br /><br />", " ").replace("<br />", ""))
src

EndPointInternalError: EndPointInternalError: The endpoint returned the HTTP status code 500. 

Response:
b'SPARQL-QUERY: queryStr=\n        SELECT ?item ?itemLabel\n        WHERE {{\n            ?item wdt:P31 wd:Q5. # \xe5\x8c\xb9\xe9\x85\x8d\xe5\xae\x9e\xe4\xbd\x93\xe7\xb1\xbb\xe5\x9e\x8b\n            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }} # \xe4\xbd\xbf\xe7\x94\xa8\xe8\x8b\xb1\xe6\x96\x87\xe6\xa0\x87\xe7\xad\xbe\n        }}\n        LIMIT 1000\n        \njava.util.concurrent.ExecutionException: java.util.concurrent.ExecutionException: java.lang.ClassCastException: com.bigdata.rdf.sparql.ast.JoinGroupNode cannot be cast to com.bigdata.rdf.sparql.ast.StatementPatternNode\n\tat java.util.concurrent.FutureTask.report(FutureTask.java:122)\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:206)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServle

In [9]:
with open("data/flair/imdb_train_flair_wiki.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(train_data):
        count = 0  # 初始化计数器
        src = replace_entities_flair_wiki(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        writer.writerow((src, p['label']))
        count+=1
        if count>5:
            break

  0%|          | 7/25000 [04:28<266:37:22, 38.40s/it]

KeyboardInterrupt



In [ ]:
cls_data = load_dataset("cnn_dailymail")
train_data = cls_data['train']
print(train_data[0])

In [ ]:
with open("data/flair/cnn_dm_train_flair_wiki.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["article","highlights"])
    for p in tqdm(train_data):
        src = replace_entities_flair_wiki(p['article'])
        trg = replace_entities_flair_wiki(p['highlights'])
        writer.writerow((src, trg))

## Spacy wiki

In [13]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [14]:
'''
#找不到anonymizer
def replace_entities_spacy_wiki(text):
    parsed = nlp(text)
    # iterate over entities and print
    replacements = []
    replacement_map = {}
    if all([w.ent_type == 0 for w in parsed]):
        return text

    for word in parsed:
        if word.text in replacement_map:
            replacement_map["-"] = "-"
            replacements.append((word.idx, word.idx + len(word.text), replacement_map[word.text], word.text))
            continue
        if word.ent_type_ == "ORG":
            repl = org.handle(word.text.split(" "), NECache())
            if not repl or " ".join(repl) == word.text:
                repl = org.handle(word.text.split(" "), NECache())
            if not repl or " ".join(repl) == word.text:
                continue
            replacements.append((word.idx, word.idx + len(word.text), " ".join(repl), word.text))
            replacement_map[word.text] = " ".join(repl)
        elif word.ent_type_ == "PERSON":
            repl = person.handle(word.text.split(" "), NECache())
            if not repl or " ".join(repl) == word.text:
                repl = person.handle(word.text.split(" "), NECache())
            if not repl or " ".join(repl) == word.text:
                continue
            replacements.append((word.idx, word.idx + len(word.text), " ".join(repl), word.text))
            replacement_map[word.text] = " ".join(repl)
        elif word.ent_type_ == "GPE":
            repl = gpe.handle([word.text], {})
            if not repl or " ".join(repl) == word.text:
                repl = gpe.handle([word.text], {})
            if not repl or " ".join(repl) == word.text:
                continue
            replacements.append((word.idx, word.idx + len(word.text), " ".join(repl), word.text))
            replacement_map[word.text] = " ".join(repl)
            replacement_map["-"] = "-"

    if replacements:
        res = []
        i = 0
        for (start, end, txt, orig) in replacements:
            assert orig != txt
            res.append(text[i:start] + txt)
#             print("\"" + text[i:start] + "\"", "\"" + orig + "\"", "\"" + txt + "\"")
            i = end
        res.append(text[end:])
        return ''.join(res)
    return text
'''

In [ ]:
def replace_entities_spacy_wiki(text):
    parsed = nlp(text)  # 使用预定义的nlp模型对文本进行处理
    replacements = []  # 存储需要替换的实体信息
    replacement_map = {}  # 存储已替换的实体映射

    if all([w.ent_type == 0 for w in parsed]):  # 如果没有命名实体
        return text  # 直接返回原始文本

    for word in parsed:
        if word.text in replacement_map:  # 如果实体已被替换过
            replacement_map["-"] = "-"  # 添加一个占位符
            # 将替换信息添加到replacements
            replacements.append((word.idx, word.idx + len(word.text), replacement_map[word.text], word.text))
            continue  # 继续处理下一个实体

        # 根据实体类型查询Wikidata获取替换文本
        if word.ent_type_ == "ORG":
            repl = query_wikidata(word.text, "ORG")
        elif word.ent_type_ == "PERSON":
            repl = query_wikidata(word.text, "PERSON")
        elif word.ent_type_ == "GPE":
            repl = query_wikidata(word.text, "GPE")
        else:
            continue  # 对其他类型实体不进行替换

        # 如果查询结果不为空且不等于原实体文本
        if repl and repl != word.text:
            # 将替换信息添加到replacements和replacement_map
            replacements.append((word.idx, word.idx + len(word.text), repl, word.text))
            replacement_map[word.text] = repl

    replacement_map["-"] = "-"  # 添加一个占位符

    if replacements:  # 如果有需要替换的实体
        res = []  # 存储替换后的文本
        i = 0  # 当前位置指针
        for start, end, txt, orig in replacements:  # 遍历每个替换信息
            assert orig != txt  # 确保替换文本与原文本不同
            res.append(text[i:start] + txt)  # 将位置指针前的原文本加上替换文本
            i = end  # 更新位置指针
        res.append(text[end:])  # 添加最后一段原文本
        return ''.join(res)  # 将替换后的文本拼接并返回

    return text  # 如果没有需要替换的实体,返回原始文本

In [15]:
cls_data = load_dataset("imdb")
train_data = cls_data['train']
print(train_data[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [16]:
with open("data/spacy/imdb_train_spacy_wiki.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(train_data):
        src = replace_entities_spacy_wiki(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        writer.writerow((src, p['label']))

  0%|          | 0/25000 [00:00<?, ?it/s]


NameError: name 'gpe' is not defined

In [ ]:
with open("data/spacy/imdb_train_spacy_wiki.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["text","label"])
    for p in tqdm(train_data):
        src = replace_entities_spacy_wiki(p['text'].replace("<br /><br />", " ").replace("<br />", ""))
        writer.writerow((src, p['label']))

In [ ]:
cls_data = load_dataset("cnn_dailymail")
train_data = cls_data['train']
print(train_data[0])

In [ ]:
with open("data/spacy/cnn_dm_train_spacy_wiki.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["article","highlights"])
    for p in tqdm(train_data):
        src = replace_entities_spacy_wiki(p['article'])
        trg = replace_entities_spacy_wiki(p['highlights'])
        writer.writerow((src, trg))